In [1]:
import sys
import os
import pandas as pd

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

In [2]:
from tqdm.auto import tqdm

object_types = [
    aq.ObjectType.find_by_name('Fragment Stock')
]

def get_items(x, object_types, sess=None):
    sess = sess or x.session
    object_type_ids = []
    for ot in object_types:
        if isinstance(ot, str):
            otid = sess.ObjectType.find_by_name(ot).id
        elif isinstance(ot, int):
            otid = ot
        else:
            otid = ot.i
        object_type_ids.append(otid)
    assert object_type_ids
    items = sess.Item.where({'sample_id': x.id, 'object_type_id': object_type_ids})
    return items
        
def filter_items_available(items):
    return [i for i in items if i.location != 'deleted']
    



with aq.with_cache(timeout=50) as sess:
    
    fragtype = sess.SampleType.find_by_name('Fragment')

    frags = sess.Sample.where('user_id = 66 AND sample_type_id = {} AND id > 36000'.format(
        fragtype.id,
    ))

        
    rows = []
    for f in tqdm(frags):
        items = get_items(f, object_types=['Fragment Stock'], sess=sess)
        available_items = filter_items_available(items)
        rows.append({
            'name': f.name,
            'has_available_items': len(available_items) > 0,
            'had_items': len(items) > 0
        })
        
df = pd.DataFrame(rows)


  0%|          | 0/73 [00:00<?, ?it/s]

In [3]:
pd.set_option('display.max_rows', 500)
df

,name,has_available_items,had_items
0,DNA MIX (TRP<7XTetO-pMinCyc1-URGR-W20-tCyc1>),False,False
1,(TRUSTED) pMOD8 backbone 1,False,False
2,TRP<7XTetO-pMinCyc1-URGR-W5-tCyc1>,False,False
3,TRP<7XTetO-pMinCyc1-URGR-W34-tCyc1>,False,False
4,3'-hom-LEU,True,True
5,3'-hom-LEU (copy),False,False
6,tCyc1(partial)-TS-TRP_3'hom__v3,True,True
7,tCyc1(partial)-TS-URA_3'hom__v3,True,True
8,tCyc1(partial)-TS-HIS_3'hom__v3,True,True
9,TS-HO_3'hom__v3,True,True


In [4]:
data = dict(zip(df.name, df.has_available_items))

In [5]:
import json

path = 'build_permutations_2021-03-08T10:16:09.773080-08:00.json'
 
with open(path, 'r') as f:
    buildjson = json.load(f)

rows = []
for x in buildjson:
    for part in x['parts']:
        sample_name = part.get('sample', dict()).get('name', '')
        if sample_name in data:
            row = {
                'name': x['name'],
                'permutation': x['permutation'],
                'fragment': sample_name,
                'ready': data[sample_name]
            }
            rows.append(row)
            
df = pd.DataFrame(rows)
df.head()

,name,permutation,fragment,ready
0,DSGRN NOR (BAD),0,HIS-HIS<7XTetO-pMinCyc1-URGR-W8>,False
1,DSGRN NOR (BAD),0,URA-URA<pGRR-W10W8-URGR-W5>,True
2,DSGRN NOR (BAD),0,ARS314-bleo<pGRR-W5W5-URGR-W20>,True
3,DSGRN NOR (BAD),0,LEU-LEU<pGRR-W5W5-URGR-W36>,True
4,DSGRN NOR (BAD),0,URA-URA<pGRR-W20W36-yeGFP>,True


In [6]:
circuit_dict = {}

for _, row in df.iterrows():
    circuit_dict.setdefault(row ['name'], {})
    circuit_dict[row ['name']].setdefault(row.permutation, [])
    if row.ready:
        circuit_dict[row ['name']][row.permutation].append(row.fragment)
        
ranked = {}

for name, d in circuit_dict.items():
    perms = list(d.items())
    perms.sort(key=lambda x: len(x[1]), reverse=True)
    ranked[name] = perms

rows = []
for name, perms in ranked.items():
    row = {
        'name': name,
        'permutation': perms[0][0],
        'available_transformations': perms[0][1]
    }
    rows.append(row)
rows

[{'name': 'DSGRN NOR (BAD)',
  'permutation': 0,
  'available_transformations': ['URA-URA<pGRR-W10W8-URGR-W5>',
   'ARS314-bleo<pGRR-W5W5-URGR-W20>',
   'LEU-LEU<pGRR-W5W5-URGR-W36>',
   'URA-URA<pGRR-W20W36-yeGFP>']},
 {'name': 'DSGRN NOR (GOOD)',
  'permutation': 0,
  'available_transformations': ['URA-URA<pGRR-W8W34-URGR-W17>',
   'ARS314-bleo<pGRR-W17W17-URGR-W10>',
   'LEU-LEU<pGRR-W17W17-URGR-W20>',
   'URA-URA<pGRR-W10W20-yeGFP>']},
 {'name': 'DSGRN OR (BAD)',
  'permutation': 0,
  'available_transformations': ['URA-URA<pGRR-W10W8-URGR-W5>',
   'ARS314-bleo<pGRR-W10W8-URGR-W36>']},
 {'name': 'DSGRN OR (GOOD)',
  'permutation': 0,
  'available_transformations': ['URA-URA<pGRR-W8W34-URGR-W10>',
   'ARS314-bleo<pGRR-W8W34-URGR-W17>',
   'URA-URA<pGRR-W10W17-yeGFP>']},
 {'name': 'GANDER NOR (BAD)',
  'permutation': 0,
  'available_transformations': []},
 {'name': 'GANDER NOR (GOOD)',
  'permutation': 0,
  'available_transformations': []},
 {'name': 'GANDER OR (BAD)',
  'permutation'

In [66]:
perms.sort(key=lambda x: len(x[1]), reverse=True)
perms

[(0, ['URA-URA<pGRR-W8W34-URGR-W17>']),
 (1, ['ARS314-bleo<pGRR-W8W34-URGR-W17>']),
 (2, ['ARS314-bleo<pGRR-W8W34-URGR-W17>'])]

In [7]:
summary = {}

for circuit in buildjson:
    if circuit['permutation'] == 0:
        name = circuit['name']
        matalpha = []
        mata = []
        for part in circuit['parts']:
            if part['haploid'] == 'Mat Alpha':
                x = matalpha
            elif part['haploid'] == 'Mat A':
                x = mata
            part_name = part['sample']['name']
            x.append(part_name)
        summary[circuit['name']] = {
            'MatA': mata,
            'MatAlpha': matalpha
        }

In [8]:
from pprint import pprint

pprint(summary)

{'DSGRN NOR (BAD)': {'MatA': ['pMOD-HO-KanMX-pREV1-rtTA-VPR',
                              'pMOD4G-dcas9-mxi1',
                              'HIS-HIS<7XTetO-pMinCyc1-URGR-W8>',
                              'URA-URA<pGRR-W10W8-URGR-W5>'],
                     'MatAlpha': ['pMODKan-HO-pACT1-ZEV4',
                                  'pMOD8-pGALZ4-URGR-W10',
                                  'ARS314-bleo<pGRR-W5W5-URGR-W20>',
                                  'LEU-LEU<pGRR-W5W5-URGR-W36>',
                                  'URA-URA<pGRR-W20W36-yeGFP>']},
 'DSGRN NOR (GOOD)': {'MatA': ['pMOD-HO-KanMX-pREV1-rtTA-VPR',
                               'pMOD4G-dcas9-mxi1',
                               'HIS-HIS<7XTetO-pMinCyc1-URGR-W34>',
                               'URA-URA<pGRR-W8W34-URGR-W17>'],
                      'MatAlpha': ['pMODKan-HO-pACT1-ZEV4',
                                   'pMOD8-pGALZ4-URGR-W8',
                                   'ARS314-bleo<pGRR-W17W17-URGR-W10>',
   

In [23]:
rows = []
for name, d in summary.items():
    for haploid, parts in d.items():
        row = [name, haploid] + parts
        rows.append(row)
    
max_len = max([len(x) for x in rows])
df_rows = []
for row in rows:
    parts = row[2:]
    parts_ = [r.lower() for r in parts]
    data = {
        'circuit': row[0],
        'haploid': row[1],
        'TF.1': None
    }
    for part in parts:
        if 'zev4' in part.lower() or 'rtt' in part.lower():
            data['TF.0'] = part
        elif 'dcas9' in part.lower():
            data['TF.1'] = part
        elif 'gfp' in part.lower():
            data['reporter'] = part
        elif 'gal' in part.lower():
            data['be'] = part
        elif 'tet' in part.lower():
            data['dox'] = part
        else:
            data.setdefault('pgrr', list())
            data['pgrr'].append(part)

    df_rows.append(data)
        
df = pd.DataFrame(df_rows)
df.to_clipboard()
df.head()

,circuit,haploid,TF.1,TF.0,dox,pgrr,be,reporter
0,DSGRN NOR (BAD),MatA,pMOD4G-dcas9-mxi1,pMOD-HO-KanMX-pREV1-rtTA-VPR,HIS-HIS<7XTetO-pMinCyc1-URGR-W8>,[URA-URA<pGRR-W10W8-URGR-W5>],NaN,NaN
1,DSGRN NOR (BAD),MatAlpha,None,pMODKan-HO-pACT1-ZEV4,NaN,"[ARS314-bleo<pGRR-W5W5-URGR-W20>, LEU-LEU<pGRR...",pMOD8-pGALZ4-URGR-W10,URA-URA<pGRR-W20W36-yeGFP>
2,DSGRN NOR (GOOD),MatA,pMOD4G-dcas9-mxi1,pMOD-HO-KanMX-pREV1-rtTA-VPR,HIS-HIS<7XTetO-pMinCyc1-URGR-W34>,[URA-URA<pGRR-W8W34-URGR-W17>],NaN,NaN
3,DSGRN NOR (GOOD),MatAlpha,None,pMODKan-HO-pACT1-ZEV4,NaN,"[ARS314-bleo<pGRR-W17W17-URGR-W10>, LEU-LEU<pG...",pMOD8-pGALZ4-URGR-W8,URA-URA<pGRR-W10W20-yeGFP>
4,DSGRN OR (BAD),MatA,pMOD4G-dcas9-mxi1,pMOD-HO-KanMX-pREV1-rtTA-VPR,HIS-HIS<7XTetO-pMinCyc1-URGR-W8>,[URA-URA<pGRR-W10W8-URGR-W5>],NaN,NaN


In [20]:
df_rows

[['DSGRN NOR (BAD)',
  'MatA',
  'pMOD-HO-KanMX-pREV1-rtTA-VPR',
  'pMOD4G-dcas9-mxi1',
  'HIS-HIS<7XTetO-pMinCyc1-URGR-W8>',
  'URA-URA<pGRR-W10W8-URGR-W5>',
  None],
 ['DSGRN NOR (GOOD)',
  'MatA',
  'pMOD-HO-KanMX-pREV1-rtTA-VPR',
  'pMOD4G-dcas9-mxi1',
  'HIS-HIS<7XTetO-pMinCyc1-URGR-W34>',
  'URA-URA<pGRR-W8W34-URGR-W17>',
  None],
 ['DSGRN OR (BAD)',
  'MatA',
  'pMOD-HO-KanMX-pREV1-rtTA-VPR',
  'pMOD4G-dcas9-mxi1',
  'HIS-HIS<7XTetO-pMinCyc1-URGR-W8>',
  'URA-URA<pGRR-W10W8-URGR-W5>',
  None],
 ['DSGRN OR (BAD)',
  'MatAlpha',
  'pMODKan-HO-pACT1-ZEV4',
  'pMOD8-pGALZ4-URGR-W10',
  'ARS314-bleo<pGRR-W10W8-URGR-W36>',
  'URA-URA<pGRR-W5W36-yeGFP>',
  None],
 ['DSGRN OR (GOOD)',
  'MatA',
  'pMOD-HO-KanMX-pREV1-rtTA-VPR',
  'pMOD4G-dcas9-mxi1',
  'HIS-HIS<7XTetO-pMinCyc1-URGR-W34>',
  'URA-URA<pGRR-W8W34-URGR-W10>',
  None],
 ['DSGRN OR (GOOD)',
  'MatAlpha',
  'pMODKan-HO-pACT1-ZEV4',
  'pMOD8-pGALZ4-URGR-W8',
  'ARS314-bleo<pGRR-W8W34-URGR-W17>',
  'URA-URA<pGRR-W10W17-yeGFP>',
